In [1]:
import os
import numpy as np
import pandas as pd

import joblib

from utils import get_features, text_embedding

from keras.models import load_model
from sentence_transformers import SentenceTransformer

c:\Users\82105\anaconda3\envs\cpuvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
current_path=os.getcwd().replace("\\", "/").replace("c:", "C:")
data_path=current_path+'/data/'
model_path=current_path+"/model/"

In [3]:
model = load_model(model_path+'my_model.h5')
encoder = joblib.load(model_path+'encoder.pkl')
scaler = joblib.load(model_path+'scaler.pkl')
with open(model_path+'text_model_name.txt', 'r') as file:
    text_model_name = file.read().strip()

In [4]:
label_file=data_path+'5차_10011.csv'
label=pd.read_csv(label_file, encoding= 'CP949' )

In [5]:
file_name='5ebd40eb2a2d173b73a032f6'
new_audio_path = data_path+"/5차_wav/"+file_name+'.wav'
new_audio_features = get_features(new_audio_path)
new_audio_features_df = pd.DataFrame([new_audio_features])

new_audio_features_df['wav_id']=file_name
new_audio_features_df['final_label']=label[label['wav_id']==file_name]['1번 감정'].values[0].lower()
new_audio_features_df['sentence']=label[label['wav_id']==file_name]['발화문'].values[0]

In [6]:
txt_embed = text_embedding(model_name = text_model_name)
new_audio_features_df = txt_embed.transform(new_audio_features_df)

# 특징 표준화
new_audio_features_scaled = scaler.transform(new_audio_features_df)

# 모델에 맞게 차원 확장
new_audio_features_scaled = np.expand_dims(new_audio_features_scaled, axis=2)

# 예측 수행
prediction = model.predict(new_audio_features_scaled)
predicted_label = np.argmax(prediction, axis=1)

1/1 [==============================] - 0s 148ms/step


In [7]:
num_categories = encoder.categories_[0].size  # OneHotEncoder에 의해 인코딩된 카테고리 수

# 모델의 예측 결과를 원-핫 인코딩 형식으로 변환
predicted_one_hot = np.zeros((predicted_label.size, num_categories))
predicted_one_hot[np.arange(predicted_label.size), predicted_label] = 1

# 원-핫 인코딩된 예측 결과를 실제 레이블로 변환
actual_label = encoder.inverse_transform(predicted_one_hot)


In [8]:
actual_label[0][0]

'angry'

In [9]:
# 출력 형식 설정
np.set_printoptions(precision=6, suppress=True)

In [10]:
prediction*100

array([[70.4457  ,  0.002099,  0.124698,  0.000014, 29.42748 ,  0.      ]],
      dtype=float32)